In [1]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://darshith.netlify.app/")
links

['https://darshith.netlify.app/#about',
 'https://darshith.netlify.app/#projects',
 'https://darshith.netlify.app/#skills',
 'https://darshith.netlify.app/#contact',
 'https://darshith.netlify.app/',
 'https://darshith.netlify.app/',
 'https://darshith.netlify.app/',
 'https://darshith.netlify.app/']

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://darshith.netlify.app/"))


Here is the list of links on the website https://darshith.netlify.app/ -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://darshith.netlify.app/#about
https://darshith.netlify.app/#projects
https://darshith.netlify.app/#skills
https://darshith.netlify.app/#contact
https://darshith.netlify.app/
https://darshith.netlify.app/
https://darshith.netlify.app/
https://darshith.netlify.app/


In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://darshith.netlify.app/")

{'links': [{'type': 'about page',
   'url': 'https://darshith.netlify.app/#about'},
  {'type': 'projects page', 'url': 'https://darshith.netlify.app/#projects'},
  {'type': 'contact page', 'url': 'https://darshith.netlify.app/#contact'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [11]:
print(fetch_page_and_all_relevant_links("https://www.lockheedmartin.com/"))

Selecting relevant links for https://www.lockheedmartin.com/ by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Lockheed Martin Corporation: Leading Aerospace and Defense
Who we are
What we do
News
Careers
Investors
Suppliers
Global Activity
United States | English
back
Global Activity
Featured
Global Presence, Local Impact
Learn how we are strengthening the economies, industries and communities of our global partner nations.
Australia
English
Canada
English
Français
Denmark
English
Danish
Germany
English
Deutsch
Greece
English
Ελληνικά
India
English
Israel
English
עברית
Japan
English
日本語
Latin America
English
Spanish
Portuguese
New Zealand
English
Norway
English
Norsk
Poland
English
Polski
Republic of Korea
English
한국어
Romania
English
Română
Saudi Arabia
English
عربى
Singapore
English
Spain
English
Español
Taiwan
English
United Arab Emirates
English
عربى
United Kingdom
English
United States
English
back
Search
back
Who we are
About Us
Leadership & Governance
Our Businesse

In [12]:

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [14]:
print(get_brochure_user_prompt("lockheedmartin", "https://www.lockheedmartin.com/"))

Selecting relevant links for https://www.lockheedmartin.com/ by calling gpt-5-nano
Found 25 relevant links

You are looking at a company called: lockheedmartin
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.


## Landing Page:

Lockheed Martin Corporation: Leading Aerospace and Defense
Who we are
What we do
News
Careers
Investors
Suppliers
Global Activity
United States | English
back
Global Activity
Featured
Global Presence, Local Impact
Learn how we are strengthening the economies, industries and communities of our global partner nations.
Australia
English
Canada
English
Français
Denmark
English
Danish
Germany
English
Deutsch
Greece
English
Ελληνικά
India
English
Israel
English
עברית
Japan
English
日本語
Latin America
English
Spanish
Portuguese
New Zealand
English
Norway
English
Norsk
Poland
English
Polski
Republic of Korea
English
한국어
Romania
English
Română
Saudi Arabia
Eng

In [15]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
(create_brochure("lockheedmartin", "https://www.lockheedmartin.com/"))

Selecting relevant links for https://www.lockheedmartin.com/ by calling gpt-5-nano
Found 9 relevant links


# Lockheed Martin: Aerospace & Defense with a Dash of Awesome

---

## Who We Are  
Lockheed Martin isn’t just a company — we’re *the* company leading the charge into the skies, beyond the clouds, and into the future. With a global presence stretching from the Outback to the Arctic, we’re the aerospace and defense heavyweight that keeps the world spinning securely — and innovatively.

---

## What We Do  
If it *flies*, *defends*, or *explores* the unknown, chances are we’re behind it. Fighter jets? Check. Cyber defenses? Double check. Space exploration technology? You bet. We are the masterminds turning science fiction into science fact through our game-changing aircraft, missile systems, AI, and space tech — all wrapped up neatly with sustainability and ethics.

**Our secret sauce?**  
- **Aeronautics:** Fighter jets, tactical airlift, and yeah, even autonomous aircraft.  
- **Missiles & Fire Control:** Precision and power to keep the peace.  
- **Rotary & Mission Systems:** Helicopters like the iconic Black Hawk family that can outrun your Monday blues.  
- **Space:** From human exploration to missile defense, we’re boldly going where no company has gone before.

And our innovation playground? The legendary **Skunk Works®** — where top-secret breakthroughs become reality.

---

## Our Culture: More Than Suits & Rockets  
At Lockheed Martin, it’s not just about rocket science (though we do plenty of that). It’s about *you*, the people who make the impossible possible. We support our employees fiercely:

- Career growth that’s out of this world — from students to seasoned pros.  
- A community that values volunteerism, ethics, and giving back.  
- Diversity and inclusion that make our teams as dynamic as our tech.  
- An environment that mixes cutting-edge innovation with good old human kindness.  

Need more? We support military veterans, future STEM enthusiasts, and foster an employee-first vibe sure to make you want to clock in every day.

---

## Customers & Partners: Our Global Family  
From Australia’s sunburnt lands to the White Cliffs of Dover, Lockheed Martin partners across 20+ countries, strengthening economies, industries, and local communities with technology that matters. U.S. allies and global partners rely on us for:

- **21st Century Security®:** Resilient multi-domain networks that keep you safe in land, sea, air, cyber, and space.  
- **Global Situational Awareness:** Because knowing what’s up (literally) can save lives.  
- **AI & Autonomy:** Self-thinking, multitasking machines that do a lot of the heavy lifting so your forces can focus on the mission.

---

## Careers: Join the Future Makers  
Ready to don a jetpack or just work alongside people who do? Lockheed Martin is hiring dreamers, doers, and anyone passionate about making the world a safer place.

- **Student & Early Career Programs:** Launch your career with internships, co-ops, and mentorships.  
- **Experienced Professionals:** Take your expertise to heights you never thought possible.  
- **Innovation Hubs & Locations:** From the U.S. to global satellite offices, work where innovation never sleeps.  
- **Work-Life Balance:** Because even superheroes need a break.

Explore open jobs and see why Life@LM is the best kind of rocket fuel for your future.

---

## Sustainability & Ethics: Because Saving the Planet Matters  
Lockheed Martin takes its responsibility seriously — from reducing environmental footprints to building resilient communities. With programs supporting veterans, STEM education, and commitment to ethical conduct, we’re shaping a future that’s smart, safe, and sustainable.

---

## Fun Fact (Because Who Doesn’t Love These?)  
Our famed **Skunk Works®** was named after a remote secretive lab inspired by a wacky 'Skonk Works' in a comic strip. Humor and innovation? Check and check.

---

# Lockheed Martin:  
*Defending Today, Innovating Tomorrow, Because the Future Doesn’t Wait.*

---

**Want in?** Visit [lockheedmartin.com/careers](https://www.lockheedmartin.com/careers) to blast off your career!  
Fly with us. Protect the planet. Explore the stars.